In [ ]:
from langchain_openai import ChatOpenAI
import os
from langchain.agents import tool
from dotenv import load_dotenv

load_dotenv()

@tool
def 

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

sk-proj-TJ6i4bMNh5THXKCylN2ET3BlbkFJ08A0yVQJS04pxmGc3zrY


In [4]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.65 requires langchain-community<0.3.0,>=0.2.16, but you have langchain-community 0.3.7 which is incompatible.
langchain-experimental 0.0.65 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.19 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
from pprint import pprint

import sqlalchemy as sa
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///wikiart-copy.db")

In [3]:
result = db.run("SELECT * FROM Artist LIMIT 12;", fetch="cursor")
print(type(result))
pprint(list(result.mappings()))

<class 'sqlalchemy.engine.cursor.CursorResult'>
[{'id': 1, 'name': 'Jeffrey Smart', 'slug': '/en/jeffrey-smart', 'wikipedia_article': 'Frank Jeffrey Edson Smart, AO (26 July 1921\xa0– 20 June 2013) was an expatriate Australian painter known for his precisionist depictions of urban landscapes that are "full of private jokes and playful allusions".\r Smart was born and educated in Adelaide where he worked as an Art teacher. After departing for Europe in 1948 he studied in Paris at La Grande Chaumière, and later at the Académie Montmartre under Fernand Léger. He returned to Australia 1951, living in Sydney, and began exhibiting frequently in 1957. In 1963, he moved to Italy. After a successful exhibition in London, he bought a rural property called "Posticcia Nuova" near Arezzo in Tuscany. He resided there with his partner until his death.\r His autobiography, Not Quite Straight, was published in 1996. A major retrospective of his works travelled around Australian art galleries 1999–2000.

# Define a SQL Chain

In [14]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from sqlalchemy import inspect
from sqlmodel import create_engine

from sqlalchemy.sql import text

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_db_schema():
    """
    Retrieve the database schema, including tables, columns, keys, and relationships.

    :param engine: SQLAlchemy engine object connected to the database
    :return: String representation of the database schema
    """
    engine = create_engine('sqlite:///wikiart-copy.db')
    inspector = inspect(engine)
    schema_details = []

    tables = inspector.get_table_names()
    schema_details.append(f"Tables: {tables}")

    for table_name in tables:
        schema_details.append(f"\nSchema for table: {table_name}")

        columns = inspector.get_columns(table_name)
        primary_keys = []
        for column in columns:
            if column.get("primary_key", False):
                primary_keys.append(column["name"])
            schema_details.append(f"  Column: {column['name']}, Type: {column['type']}, Nullable: {column['nullable']}")

        # Primary Keys
        schema_details.append(f"  Primary Keys: {primary_keys}")

        foreign_keys = inspector.get_foreign_keys(table_name)
        if foreign_keys:
            schema_details.append("  Foreign Keys:")
            for fk in foreign_keys:
                schema_details.append(
                    f"    {fk['constrained_columns']} -> {fk['referred_table']}({fk['referred_columns']})"
                )

        indexes = inspector.get_indexes(table_name)
        if indexes:
            schema_details.append("  Indexes:")
            for index in indexes:
                schema_details.append(
                    f"    Name: {index['name']}, Columns: {index['column_names']}, Unique: {index['unique']}"
                )

    return "\n".join(schema_details)


@tool
def execute_query(query: str):
    """
    Executes a SQL query on a read only database and returns the result.
    """
    engine = create_engine('sqlite:///wikiart-copy.db?mode=ro')

    import sqlmodel

    with sqlmodel.Session(engine) as session:
        result = session.exec(text(query))
        return result.fetchall()


llm_with_tools = llm.bind_tools([get_db_schema, execute_query])
llm_with_tools.invoke("List an artist in the database").tool_calls


# chain = llm_with_tools | (lambda x: x.tool_calls[0]['args']) | get_db_schema
# res = chain.invoke("Could you list a random artist from the database?")
# print(res)

[{'name': 'execute_query',
  'args': {'query': 'SELECT name FROM artists LIMIT 1;'},
  'id': 'call_lfH9K1TbcMnsJK0cUxe04De2',
  'type': 'tool_call'}]

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

tools = [get_db_schema, execute_query]
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an automated art guide. You will use your provided SQL tools to redirect the human."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor = agent_executor.invoke({"input": "List an artist in the database."})



> Entering new AgentExecutor chain...

Invoking: `get_db_schema` with `{}`


Tables: ['artist', 'artistartmovement', 'artistfield', 'artistgenre', 'artistinstitution', 'artistnationality', 'artistschool', 'artwork', 'artworkgenre', 'artworkmedia', 'artworkstyle']

Schema for table: artist
  Column: id, Type: INTEGER, Nullable: False
  Column: name, Type: VARCHAR, Nullable: True
  Column: slug, Type: VARCHAR, Nullable: False
  Column: wikipedia_article, Type: VARCHAR, Nullable: True
  Column: wikipedia_url, Type: VARCHAR, Nullable: True
  Column: article, Type: VARCHAR, Nullable: True
  Primary Keys: ['id']
  Indexes:
    Name: ix_artist_slug, Columns: ['slug'], Unique: 0

Schema for table: artistartmovement
  Column: id, Type: INTEGER, Nullable: False
  Column: artist_id, Type: INTEGER, Nullable: False
  Column: art_movement, Type: VARCHAR, Nullable: False
  Primary Keys: ['id']
  Foreign Keys:
    ['artist_id'] -> artist(['id'])

Schema for table: artistfield
  Column: id, Type: INT

In [26]:
@tool
def get_image_from_url(url: str):
    """
    Gets an artwork image from its URL.
    """
    import requests
    from bs4 import BeautifulSoup as bs

    url = f"https://wikiart.org{url}"
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    img = soup.find("img", {"itemprop": "image"})
    display(Image(url=img.get("src")))

In [28]:
get_image_from_url("/en/jeffrey-smart/water-towers-1944")

In [ ]:
while True:
    # Agent loop
    

In [ ]:
import ipywidgets as widgets

## LightRag

RAG systems enhance LLMs by integrating external knowledge sources. Traditional RAG solutions assume a 'flat' data representation, which can fail to capture complex dependencies. 
We use a hierarchical representation a knowledge graph to model our inherently hierarchical artwork data. 

LightRAG incorporates graph structures into indexing and retrieval. It creates a knowledge graph encoding 'entities' and their relationships, using techniques such as entity recognition, relation extration, and graph embeding models. It's dual-level retrieval system is perfect for our Artwork dataset. Fine grained details like artist names, artwork titles, and their references are retrieved via low-level concepts, and thematic information is retrieved via high-level concepts.

We provide a dataset to consume and create your own LightRAG database. The dataset is a collection of artwork metadata scraped from [wikiart](https://www.wikiart.org/). It includes information about artists such as their wikipedia articles